In [1]:
import os
import torch
from threading import Thread
from transformers import AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer, AutoModel

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
MODEL_PATH = os.environ.get('MODEL_PATH', '/data/zhenyu/LLM_Model/GLM_4/GLM4_Chat/')

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    encode_special_tokens=True
)

model = AutoModel.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    # attn_implementation="flash_attention_2", # Use Flash Attention
    # torch_dtype=torch.bfloat16, #using flash-attn must use bfloat16 or float16
    device_map="auto").eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = model.config.eos_token_id
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def predict(messages):
    stop = StopOnTokens()
    model_inputs = tokenizer.apply_chat_template(messages,
                                                 add_generation_prompt=True,
                                                 tokenize=True,
                                                 return_tensors="pt").to(next(model.parameters()).device)
    streamer = TextIteratorStreamer(tokenizer, timeout=60, skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = {
        "input_ids": model_inputs,
        "streamer": streamer,
        "max_new_tokens": 512,
        "do_sample": True,
        "top_p": 0.8,
        "temperature": 0.1,
        "stopping_criteria": StoppingCriteriaList([stop]),
        "repetition_penalty": 1.2,
        "eos_token_id": model.config.eos_token_id,
    }
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()
    result = ''
    for new_token in streamer:
        result += new_token
        print(new_token, end='')
    return result


In [5]:
system_prompt = f"""
你是一个经验丰富的呼吸科医生，擅长基于患者CAC（循环染色体异常细胞）与CTAI的检查结果进行分析与解读。以下是针对不同类别患者的参考意见：\
```
情况1： CAC > 9, CTAI 结节风险 > 85%；分析解读参考: CAC、CTAI均在高风险区间，应选择积极治疗，特别是结节大小≥8mm，手术干预指征更强。如果结节＜8mm，或患者拒绝手术治疗时，可选择密切观察（每3个月复查胸部CT），随访过程中如有结节增大、密度增高、实性成分增加等，需再次建议患者积极治疗（手术切除）。当然，尽管CAC大于9，CTAI＞85%，提示肺结节恶性风险高危，但手术切除后病理依然存在良性结节的可能，虽然这种可能性比较小。
情况2： CAC > 9, CTAI 结节风险 < 85%；分析解读参考: CAC高风险区间，CTAI未达高风险区间，建议抗炎治疗（两周）后3个月复查胸部CT，同时进行CTAI随访对比。如有结节增大、密度增高、实性成分增加等，可建议积极治疗，如果没有结节增大、密度增高、实性成分增加等，应对患者进行密切随访，每3个月复查胸部CT，连续3-4次，随访观察过程中出现结节恶性倾向增大（结节增大、密度增高、实性成分增加或出现下列征象一项以上时：结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等，建议积极治疗。
情况3： 3 ≤ CAC ≤ 9, CTAI 结节风险 > 85%；分析解读参考: CAC中高风险区间，CTAI高风险区间，应选择积极治疗。特别是结节大小≥8mm，手术干预指征更强，如果结节＜8mm，或患者拒绝手术治疗时，可选择密切观察（每3个月复查胸部CT），随访过程中如有结节增大、密度增高、实性成分增加等，需再次建议患者积极治疗（手术切除）。
情况4： 3 ≤ CAC ≤ 9, CTAI 结节风险 < 85%；分析解读参考：CAC中高风险区间，CTAI未达高风险区间，应对患者进行定期随访，3-6个月复查胸部CT，同时CTAI进行随访对比分析。
情况5： CAC < 3, CTAI 结节风险 > 85%；分析解读参考：CAC低风险区间，CTAI高风险区间，建议密切随访观察。每3个月复查胸部CT，同时进行CTAI随访对比，如有结节增大、密度增高、实性成分增加等，可建议积极治疗；如果没有结节增大、密度增高、实性成分增加等，应继续对患者进行密切随访，每3个月复查胸部CT，连续3-4次，随访观察过程中出现结节恶性倾向增大（结节增大、密度增高、实性成分增加或出现下列征象一项以上时：结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等，建议积极治疗。
情况6： CAC < 3, CTAI 结节风险 < 85%；分析解读参考：良性结节的可能性较大，建议年度体检，12个月复查胸部CT。如果结节大于15mm，或患者焦虑明显、要求治疗意愿明显，可考虑手术干预，但术后病理良性结节概率较高。
```
请依据上述信息，确认患者属于哪个情况。然后解读患者的CAC与CTAI检测结果，最后给患者提炼‘分析解读参考’中合适的建议。
请简短回答。
"""

In [32]:
messages_1 = []
messages_1.append({"role":"system", "content":system_prompt})
check_info_1 = """
患者姓名：尹玉梅；
CAC: 20;
CTAI: 
1号结节 恶性风险等级：90%；结节类型：混合型；结节大小：15mm, 
2号结节 恶行风险等级：80%；结节类型：磨玻璃型；结节大小：6mm；
"""
messages_1.append({"role":"user", "content":check_info_1})
predict(messages_1)


患者属于情况1。

分析解读：
尹玉梅女士的CAC值为20，远高于正常范围，表明其肺部结节的恶性风险很高。两个结节的CTAI风险等级分别为90%和80%，且均为高危险级别。其中，1号结节为混合型，直径达到15mm，已超过通常认为需要治疗的阈值（一般认为是8mm及以上）；2号结节虽较小（6mm），但其恶性行风险的CTAI评分也达到了80%。这些数据都强烈支持采取积极的医疗措施。

建议：
鉴于CAC值极高以及两个结节的高危CTAI评分，尤其是大结节的大小，尹玉梅女士应该接受积极的治疗，包括手术切除。对于1号结节，由于它已经很大并且具有很高的恶性风险，立即手术是必要的。对于2号结节，由于其较小的尺寸和高风险评分，也应考虑早期手术以预防潜在的恶化。在手术前可能需要进行进一步的评估和准备。如果患者因任何原因不愿意或不适合手术，则应根据具体情况制定其他治疗方案。

'\n患者属于情况1。\n\n分析解读：\n尹玉梅女士的CAC值为20，远高于正常范围，表明其肺部结节的恶性风险很高。两个结节的CTAI风险等级分别为90%和80%，且均为高危险级别。其中，1号结节为混合型，直径达到15mm，已超过通常认为需要治疗的阈值（一般认为是8mm及以上）；2号结节虽较小（6mm），但其恶性行风险的CTAI评分也达到了80%。这些数据都强烈支持采取积极的医疗措施。\n\n建议：\n鉴于CAC值极高以及两个结节的高危CTAI评分，尤其是大结节的大小，尹玉梅女士应该接受积极的治疗，包括手术切除。对于1号结节，由于它已经很大并且具有很高的恶性风险，立即手术是必要的。对于2号结节，由于其较小的尺寸和高风险评分，也应考虑早期手术以预防潜在的恶化。在手术前可能需要进行进一步的评估和准备。如果患者因任何原因不愿意或不适合手术，则应根据具体情况制定其他治疗方案。'

In [33]:
messages_2 = []
messages_2.append({"role":"system", "content":system_prompt})
check_info_2 = """
患者姓名：弓淑琴；
CAC: 3;
CTAI: 
1号结节 恶性风险等级：90%；结节类型：磨玻璃型；结节位置：右肺下叶；结节大小：15mm, 
2号结节 恶性风险等级：90%；结节类型：磨玻璃型；结节位置：左肺上叶；结节大小：6mm；
3号结节 恶性风险等级：88%；结节类型：混合型；结节位置：右肺下叶；结节大小：4mm, 
"""
messages_2.append({"role":"user", "content":check_info_2})
predict(messages_2)


患者属于情况5。

解析：
- CAC为3，处于低风险区间。
- 所有三个结节的CTAI恶性风险等级都超过85%。

建议：
- 密切随访观察，每3个月复查胸部CT。
- 如有结节增大、密度增高、实性成分增加等变化，建议积极治疗。
- 随访过程中如发现结节恶性倾向增大（包括结节增大、密度增高、实性成分增加或出现分叶、毛刺、胸膜牵拉、血管集束、空泡征等），建议积极治疗。
- 由于两个大结节（15mm和6mm）的恶性风险高且体积较大，可能需要更积极的监测和治疗策略，甚至考虑早期手术干预。

'\n患者属于情况5。\n\n解析：\n- CAC为3，处于低风险区间。\n- 所有三个结节的CTAI恶性风险等级都超过85%。\n\n建议：\n- 密切随访观察，每3个月复查胸部CT。\n- 如有结节增大、密度增高、实性成分增加等变化，建议积极治疗。\n- 随访过程中如发现结节恶性倾向增大（包括结节增大、密度增高、实性成分增加或出现分叶、毛刺、胸膜牵拉、血管集束、空泡征等），建议积极治疗。\n- 由于两个大结节（15mm和6mm）的恶性风险高且体积较大，可能需要更积极的监测和治疗策略，甚至考虑早期手术干预。'

In [31]:
messages_3 = []
messages_3.append({"role":"system", "content":system_prompt})
check_info_3 = """
患者姓名：弓淑琴；
CAC: 5;
CTAI: 
1号结节 恶性风险等级：23%；结节类型：实性结节；结节位置：右肺上叶；
"""
messages_3.append({"role":"user", "content":check_info_3})
predict(messages_3)


根据提供的信息：

**患者所属情况：**
情况4：3 ≤ CAC ≤ 9, CTAI 结节风险 < 85%

**分析解读参考：**

患者弓淑琴的CAC为5，处于中等风险区间，而1号结节的恶行风险等级为23%，低于85%。由于CAC在中风险范围且CTAI结节风险也较低，建议对弓淑琴进行以下处理：

* 定期随访：每隔3至6个月复查一次胸部CT扫描，以监测结节的变化。
* 密切观察：在随访期间密切关注结节的大小、形态和密度变化。
* 如有恶化迹象：若发现结节增大、密度增高或有其他恶性特征如分叶、毛刺等，应及时评估并可能需要进一步的诊断和治疗。

**具体建议：**
建议弓淑琴每3到6个月进行一次胸部CT扫描，持续监测右肺上叶的实性结节。若无任何恶化迹象，则继续保持密切随访。若有任何可疑变化，及时咨询医生并进行进一步评估。

'\n根据提供的信息：\n\n**患者所属情况：**\n情况4：3 ≤ CAC ≤ 9, CTAI 结节风险 < 85%\n\n**分析解读参考：**\n\n患者弓淑琴的CAC为5，处于中等风险区间，而1号结节的恶行风险等级为23%，低于85%。由于CAC在中风险范围且CTAI结节风险也较低，建议对弓淑琴进行以下处理：\n\n* 定期随访：每隔3至6个月复查一次胸部CT扫描，以监测结节的变化。\n* 密切观察：在随访期间密切关注结节的大小、形态和密度变化。\n* 如有恶化迹象：若发现结节增大、密度增高或有其他恶性特征如分叶、毛刺等，应及时评估并可能需要进一步的诊断和治疗。\n\n**具体建议：**\n建议弓淑琴每3到6个月进行一次胸部CT扫描，持续监测右肺上叶的实性结节。若无任何恶化迹象，则继续保持密切随访。若有任何可疑变化，及时咨询医生并进行进一步评估。'

In [34]:
messages_4 = []
messages_4.append({"role":"system", "content":system_prompt})
check_info_4 = """
患者姓名：张学阳；
CAC: 0;
CTAI: 
所有结节 恶行风险概率：1%；
"""
messages_4.append({"role":"user", "content":check_info_4})
predict(messages_4)


患者属于情况6。

解析：
- CAC为0，表示循环染色体异常细胞的数量很低，处于低风险区间。
- 所有结节的恶行风险概率为1%，低于85%，说明CTAI也处于低风险区间。

根据“分析解读参考”的建议：
- 建议张学阳先生进行年度体检，并每隔12个月复查一次胸部CT。
- 如果结节直径超过15毫米，或者患者表现出明显的焦虑和强烈的治疗愿望，可以考虑手术干预。
- 由于术后病理显示良性结节的概率较高，因此在这种情况下，通常不会立即推荐积极的手术治疗。

'\n患者属于情况6。\n\n解析：\n- CAC为0，表示循环染色体异常细胞的数量很低，处于低风险区间。\n- 所有结节的恶行风险概率为1%，低于85%，说明CTAI也处于低风险区间。\n\n根据“分析解读参考”的建议：\n- 建议张学阳先生进行年度体检，并每隔12个月复查一次胸部CT。\n- 如果结节直径超过15毫米，或者患者表现出明显的焦虑和强烈的治疗愿望，可以考虑手术干预。\n- 由于术后病理显示良性结节的概率较高，因此在这种情况下，通常不会立即推荐积极的手术治疗。'

In [7]:
messages_2 = []
messages_2.append({"role":"system", "content":system_prompt})
check_info_2 = """
患者姓名：刘贵玲；
CAC: 12;
CTAI: 
1号结节 恶性风险等级：57%；结节类型：磨玻璃结节；结节大小：6.4mm；
"""
messages_2.append({"role":"user", "content":check_info_2})
predict(messages_2)


患者属于情况5。

解析：
- CAC为12，处于低风险区间（<3）。
- 1号结节的CTAI恶性风险等级为57%，低于85%的高风险阈值。

建议：
- 密切随访观察，每3个月复查胸部CT。
- 进行CTAI随访对比，监测结节变化。
- 如果发现结节增大、密度增高、实性成分增加或有以下恶性行为特征之一：结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等，建议积极治疗。
- 如无上述改变，持续随访，连续3-4次复查未见恶化趋势，则可能保持现状。

'\n患者属于情况5。\n\n解析：\n- CAC为12，处于低风险区间（<3）。\n- 1号结节的CTAI恶性风险等级为57%，低于85%的高风险阈值。\n\n建议：\n- 密切随访观察，每3个月复查胸部CT。\n- 进行CTAI随访对比，监测结节变化。\n- 如果发现结节增大、密度增高、实性成分增加或有以下恶性行为特征之一：结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等，建议积极治疗。\n- 如无上述改变，持续随访，连续3-4次复查未见恶化趋势，则可能保持现状。'

In [10]:
messages_3 = []
messages_3.append({"role":"system", "content":system_prompt})
check_info_3 = """
患者姓名：衡凤英；
CAC: 2;
CTAI: 
1号结节 结节风险：56%；结节类型：实性结节；结节大小：8.7mm；
2号结节 结节风险：61%；结节类型：混合型结节；结节大小：5.5mm；
"""
messages_3.append({"role":"user", "content":check_info_3})
predict(messages_3)


患者属于情况5。

分析解读：
患者CAC为2，处于低风险区间；两个结节的CTAI分别为56%和61%，都低于85%。第一个结节是实性结节，大小8.7mm；第二个结节是混合型结节，大小5.5mm。

建议：
由于CAC低风险且两个结节的风险也较低，建议对衡凤英女士进行密切随访观察。具体包括每3个月复查一次胸部CT，并持续监测结节的变化。如果有以下任何一种情况发生，如结节增大、密度增高、实性成分增加，或者出现结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等恶性倾向增大的迹象，则应及时建议进一步的治疗。在没有这些变化的情况下，可以继续保持密切随访。对于小于8mm的小结节，通常不需要立即处理，除非有其他临床担忧。

'\n患者属于情况5。\n\n分析解读：\n患者CAC为2，处于低风险区间；两个结节的CTAI分别为56%和61%，都低于85%。第一个结节是实性结节，大小8.7mm；第二个结节是混合型结节，大小5.5mm。\n\n建议：\n由于CAC低风险且两个结节的风险也较低，建议对衡凤英女士进行密切随访观察。具体包括每3个月复查一次胸部CT，并持续监测结节的变化。如果有以下任何一种情况发生，如结节增大、密度增高、实性成分增加，或者出现结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等恶性倾向增大的迹象，则应及时建议进一步的治疗。在没有这些变化的情况下，可以继续保持密切随访。对于小于8mm的小结节，通常不需要立即处理，除非有其他临床担忧。'

In [11]:
messages_4 = []
messages_4.append({"role":"system", "content":system_prompt})
check_info_4 = """
患者姓名：张学阳；
CAC: 0;
CTAI: 
1号结节 结节风险：0%；结节类型：实性结节；结节大小：8.7mm；
2号结节 结节风险：1%；结节类型：混合型结节；结节大小：9mm；
"""
messages_4.append({"role":"user", "content":check_info_4})
predict(messages_4)


患者属于情况6。

解析：
- 患者的CAC为0，处于低风险区间。
- 对于两个结节，第一个结节的CTAI结节风险为0%，第二个结节的CTAI结节风险为1%。两者都低于85%的高风险阈值。

建议：
- 良性结节的可能性较大，建议年度体检，12个月复查胸部CT。
- 如果结节在随诊期间无变化且小于15mm，可以继续保持监测状态。
- 若结节超过15mm或者患者有明显的焦虑和强烈的治疗愿望，可以考虑手术干预，因为术后病理显示良性结节的概率较高。

'\n患者属于情况6。\n\n解析：\n- 患者的CAC为0，处于低风险区间。\n- 对于两个结节，第一个结节的CTAI结节风险为0%，第二个结节的CTAI结节风险为1%。两者都低于85%的高风险阈值。\n\n建议：\n- 良性结节的可能性较大，建议年度体检，12个月复查胸部CT。\n- 如果结节在随诊期间无变化且小于15mm，可以继续保持监测状态。\n- 若结节超过15mm或者患者有明显的焦虑和强烈的治疗愿望，可以考虑手术干预，因为术后病理显示良性结节的概率较高。'

In [12]:
messages_5 = []
messages_5.append({"role":"system", "content":system_prompt})
check_info_5 = """
患者姓名：张凤玲；
CAC: 0;
CTAI: 
1号结节 结节风险：89%；结节类型：混合型结节；结节大小：8.7mm；
"""
messages_5.append({"role":"user", "content":check_info_5})
predict(messages_5)


患者属于情况5。

解析：
- 患者的CAC为0，处于低风险区间。
- CTAI显示1号结节的结节风险为89%，达到高风险区间。

根据情况5的分析解读参考：
建议密切随访观察。每3个月复查胸部CT，同时进行CTAI随访对比。如有结节增大、密度增高、实性成分增加等，可建议积极治疗；如果没有这些变化，应继续对患者进行密切随访，每3个月复查胸部CT，连续3-4次。在随访观察过程中，若发现结节有恶性倾向增大的迹象（如结节增大、密度增高、实性成分增加或出现结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等），则建议积极治疗。

'\n患者属于情况5。\n\n解析：\n- 患者的CAC为0，处于低风险区间。\n- CTAI显示1号结节的结节风险为89%，达到高风险区间。\n\n根据情况5的分析解读参考：\n建议密切随访观察。每3个月复查胸部CT，同时进行CTAI随访对比。如有结节增大、密度增高、实性成分增加等，可建议积极治疗；如果没有这些变化，应继续对患者进行密切随访，每3个月复查胸部CT，连续3-4次。在随访观察过程中，若发现结节有恶性倾向增大的迹象（如结节增大、密度增高、实性成分增加或出现结节分叶、毛刺、胸膜牵拉、血管集束、空泡征等），则建议积极治疗。'

In [13]:
messages_6 = []
messages_6.append({"role":"system", "content":system_prompt})
check_info_6 = """
患者姓名：陈德良；
CAC: 4;
CTAI: 
1号结节 结节风险：93%；结节类型：混合型结节；结节大小：15mm；
2号结节 结节风险：67%；结节类型：实性结节；结节大小：8mm；
"""
messages_6.append({"role":"user", "content":check_info_6})
predict(messages_6)


患者属于情况5。

解析：
- 患者的CAC为4，处于低风险区间。
- 第一个结节的CTAI结节风险为93%，第二个结节的CTAI结节风险为67%，两者都高于85%的高风险阈值。

根据情况5的分析解读参考：

建议：
- 建议密切随访观察，每3个月复查胸部CT，同时进行CTAI随访对比。
- 如有结节增大、密度增高、实性成分增加等变化，可建议积极治疗。
- 如果没有这些变化，应继续对患者进行密切随访，每3个月复查一次胸部CT，连续3-4次。
- 随访过程中如发现结节恶性倾向增大的迹象（包括结节增大、密度增高、实性成分增加或出现分叶、毛刺、胸膜牵拉、血管集束、空泡征等），则建议积极治疗。

'\n患者属于情况5。\n\n解析：\n- 患者的CAC为4，处于低风险区间。\n- 第一个结节的CTAI结节风险为93%，第二个结节的CTAI结节风险为67%，两者都高于85%的高风险阈值。\n\n根据情况5的分析解读参考：\n\n建议：\n- 建议密切随访观察，每3个月复查胸部CT，同时进行CTAI随访对比。\n- 如有结节增大、密度增高、实性成分增加等变化，可建议积极治疗。\n- 如果没有这些变化，应继续对患者进行密切随访，每3个月复查一次胸部CT，连续3-4次。\n- 随访过程中如发现结节恶性倾向增大的迹象（包括结节增大、密度增高、实性成分增加或出现分叶、毛刺、胸膜牵拉、血管集束、空泡征等），则建议积极治疗。'

In [14]:
messages_7 = []
messages_7.append({"role":"system", "content":system_prompt})
check_info_7 = """
患者姓名：王改荣；
CAC: 4;
CTAI: 
1号结节 结节风险：77%；结节类型：混合型结节；结节大小：11mm，与2月22相比，风险降低，体积增加13%，密度降低22%，体积倍增时间698天；
"""
messages_7.append({"role":"user", "content":check_info_7})
predict(messages_7)


根据提供的信息，王改荣女士的情况符合以下分类：

**情况3：3 ≤ CAC ≤ 9, CTAI 结节风险 > 85%**

**分析解读参考：**
- 患者的CAC为4，处于中高风险区间。
- 1号结节的CTAI结节风险为77%，低于85%，但在逐渐接近高风险阈值。
- 虽然CTAI风险目前未达到高值，但由于结节的大小和变化趋势，需要特别关注。

**建议：**
- 由于结节大小已超过8mm且体积有所增加，建议对王女士进行积极的临床管理。
- 如果结节持续增长或有其他恶性的迹象（如密度增加），则可能需要进行进一步的评估，包括可能的活检或其他影像学检查。
- 建议每3个月复查一次胸部CT以监测结节的变化。
- 如有新的症状或结节特征恶化，应及时就医。

'\n根据提供的信息，王改荣女士的情况符合以下分类：\n\n**情况3：3 ≤ CAC ≤ 9, CTAI 结节风险 > 85%**\n\n**分析解读参考：**\n- 患者的CAC为4，处于中高风险区间。\n- 1号结节的CTAI结节风险为77%，低于85%，但在逐渐接近高风险阈值。\n- 虽然CTAI风险目前未达到高值，但由于结节的大小和变化趋势，需要特别关注。\n\n**建议：**\n- 由于结节大小已超过8mm且体积有所增加，建议对王女士进行积极的临床管理。\n- 如果结节持续增长或有其他恶性的迹象（如密度增加），则可能需要进行进一步的评估，包括可能的活检或其他影像学检查。\n- 建议每3个月复查一次胸部CT以监测结节的变化。\n- 如有新的症状或结节特征恶化，应及时就医。'

In [15]:
messages_8 = []
messages_8.append({"role":"system", "content":system_prompt})
check_info_8 = """
患者姓名：张爱霞；
CAC: 10;
CTAI: 
1号结节 结节风险：70%；结节类型：混合型结节；结节大小：11mm，较3月5日新发；
随访相同结节都为低风险
"""
messages_8.append({"role":"user", "content":check_info_8})
predict(messages_8)


患者属于情况1。

解析：
- CAC值为10，处于高风险区间。
- CTAI显示1号结节的结节风险为70%，低于85%的高风险阈值，但由于该结节为新发现且大小达到11mm，结合其混合型和随时间增大的特点，需要按照高风险处理。

建议：
- 由于CAC和CTAI均指示有较高的恶性风险，尤其是结节大小已超过8mm，建议积极治疗，包括手术干预以排除恶性肿瘤的可能性。
- 如果患者同意，应尽快安排手术切除结节。
- 若患者拒绝手术治疗，则应密切监测结节的变化，每3个月复查一次胸部CT，并注意观察结节是否增大、密度增高、实性成分增加或其他恶性的迹象如分叶、毛刺等。
- 随访期间若出现任何恶性特征，应及时调整治疗方案至积极的手术治疗。

'\n患者属于情况1。\n\n解析：\n- CAC值为10，处于高风险区间。\n- CTAI显示1号结节的结节风险为70%，低于85%的高风险阈值，但由于该结节为新发现且大小达到11mm，结合其混合型和随时间增大的特点，需要按照高风险处理。\n\n建议：\n- 由于CAC和CTAI均指示有较高的恶性风险，尤其是结节大小已超过8mm，建议积极治疗，包括手术干预以排除恶性肿瘤的可能性。\n- 如果患者同意，应尽快安排手术切除结节。\n- 若患者拒绝手术治疗，则应密切监测结节的变化，每3个月复查一次胸部CT，并注意观察结节是否增大、密度增高、实性成分增加或其他恶性的迹象如分叶、毛刺等。\n- 随访期间若出现任何恶性特征，应及时调整治疗方案至积极的手术治疗。'

In [16]:
messages_9 = []
messages_9.append({"role":"system", "content":system_prompt})
check_info_9 = """
患者姓名：弓淑琴；
CAC: 3;
CTAI: 
1号结节 结节风险：90%；结节类型：磨玻璃结节；结节大小：12mm；
2号结节 结节风险：90%；结节类型：磨玻璃结节；结节大小：8mm
3号结节 结节风险：88%；结节类型：混合型结节；结节大小：15mm
"""
messages_9.append({"role":"user", "content":check_info_9})
predict(messages_9)


患者属于情况5。

解析：
- 患者的CAC为3，处于低风险区间。
- 所有三个结节的CTAI结节风险均为高值（>85%），表明结节具有高度恶性的可能。

建议：
- 建议密切随访观察，每3个月复查胸部CT，同时进行CTAI随访对比。
- 如有结节增大、密度增高、实性成分增加等变化，应及时评估并考虑积极治疗措施。
- 由于所有结节均有较高的恶性风险，尤其是最大结节已达到15mm且为混合型结节，强烈推荐进一步临床评估和可能的活检以确定其性质。如果患者同意，可以考虑早期手术干预。

'\n患者属于情况5。\n\n解析：\n- 患者的CAC为3，处于低风险区间。\n- 所有三个结节的CTAI结节风险均为高值（>85%），表明结节具有高度恶性的可能。\n\n建议：\n- 建议密切随访观察，每3个月复查胸部CT，同时进行CTAI随访对比。\n- 如有结节增大、密度增高、实性成分增加等变化，应及时评估并考虑积极治疗措施。\n- 由于所有结节均有较高的恶性风险，尤其是最大结节已达到15mm且为混合型结节，强烈推荐进一步临床评估和可能的活检以确定其性质。如果患者同意，可以考虑早期手术干预。'

In [17]:
messages_10 = []
messages_10.append({"role":"system", "content":system_prompt})
check_info_10 = """
患者姓名：王向阳；
CAC: 5;
CTAI: 
1号结节 结节风险：23%；结节类型：实性结节；结节大小：12mm；
"""
messages_10.append({"role":"user", "content":check_info_10})
predict(messages_10)


患者属于情况4。

解析：
- 患者的CAC为5，处于3至9的中风险区间。
- CTAI显示1号结节的结节风险为23%，低于85%的高风险阈值。

建议：
- 对患者进行定期随访，建议每3-6个月复查一次胸部CT。
- 同时对CTAI进行持续监测和对比分析。
- 如果在随访期间发现结节有增大、密度增高、实性成分增加或其他可疑特征（如分叶、毛刺、胸膜牵拉、血管集束、空泡征等），应及时评估并可能需要进一步的诊断和治疗措施。

'\n患者属于情况4。\n\n解析：\n- 患者的CAC为5，处于3至9的中风险区间。\n- CTAI显示1号结节的结节风险为23%，低于85%的高风险阈值。\n\n建议：\n- 对患者进行定期随访，建议每3-6个月复查一次胸部CT。\n- 同时对CTAI进行持续监测和对比分析。\n- 如果在随访期间发现结节有增大、密度增高、实性成分增加或其他可疑特征（如分叶、毛刺、胸膜牵拉、血管集束、空泡征等），应及时评估并可能需要进一步的诊断和治疗措施。'